In [1]:
import pandas as pd
import pickle

In [2]:
strat_test_set = pd.read_csv(filepath_or_buffer='../../data/processed/test.csv')

In [3]:
X_test = strat_test_set.drop("median_house_value", axis=1)
y_test = strat_test_set["median_house_value"].copy()

In [4]:
X_test_num = X_test.drop("ocean_proximity", axis=1)
file = open('../../artifacts/imputer.pkl','rb')
imputer = pickle.load(file)
X_test_prepared = imputer.transform(X_test_num)
X_test_prepared = pd.DataFrame(
    X_test_prepared, columns=X_test_num.columns, index=X_test.index
)
X_test_prepared["rooms_per_household"] = (
    X_test_prepared["total_rooms"] / X_test_prepared["households"]
)
X_test_prepared["bedrooms_per_room"] = (
    X_test_prepared["total_bedrooms"] / X_test_prepared["total_rooms"]
)
X_test_prepared["population_per_household"] = (
    X_test_prepared["population"] / X_test_prepared["households"]
)

X_test_cat = X_test[["ocean_proximity"]]
X_test_prepared = X_test_prepared.join(pd.get_dummies(X_test_cat, drop_first=True))

In [5]:
file = open('../../artifacts/final_model.pkl','rb')
final_model = pickle.load(file)
final_predictions = final_model.predict(X_test_prepared)

In [6]:
output = pd.DataFrame()
output['Actual'] = y_test
output['Prediction'] = final_predictions 
output.to_csv(path_or_buf='../../data/processed/output.csv',index=False)